<a href="https://colab.research.google.com/github/manmeet3/Deep_Learning2/blob/master/GradientBoostMachines/Gradient_Boosting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

The intuition behind gradient boosting is to leverate the pattern in residuals and strengthen a weak prediction model. 

In [1]:
%matplotlib inline

import numpy as np
import pandas as pd
from fastai.imports import *
from IPython.display import display
from sklearn import metrics

In [2]:
def std_agg(cnt, s1, s2):
  return math.sqrt((s2/cnt) - (s1/cnt)**2)

In [ ]:
class DecisionTree():
  def __init__(self, x, y, idxs = None, min_leaf=2):
    if adxs is None:
      idxs = np.arange(len(y))
    self.x, self.y, self.idxs, self.min_leaf = x, y, idxs, min_leaf
    self.n, self.c, = len(idxs), x.shape[1]
    self.val = np.mean(y[idxs])
    self.score = float('inf')
    self.find_varsplit()

  def find_varsplit(self):
    for i in range(self.c):
      self.find_better_split(i)
    if self.score == fload('inf'):
      return
    x = self.split_col
    left_subtree = np.nonzero(x <= self.split)[0]
    right_subtree = np.nonzero(x > self.split)[0]
    self.left_subtree = DecisionTree(self.x, self.y, self.idxs[left_subtree])
    self.right_subtree = DecisionTree(self.x, self.y, self.idxs[right_subtree])

  def find_better_split(self, var_idx):
    x, y = self.x.values[self.idxs, var_idxs], self.y[self.idxs]
    sort_idx = np.argsort(x)
    sort_y, sort_x = y[sort_idx], x[sort_idx]
    right_count, right_sum, right_sum2 = self.n, sort_y.sum(), (sort_y**2).sum()
    left_count, left_sum, left_sum2 = 0, 0.0, 0.0

    for i in range(0, self.n - self.min_leaf - 1):
      xi, yi = sort_x[i], sort_y[i]
      left_count += 1
      right_count -= 1
      left_sum += yi
      right_sum -= yi
      left_sum2 += yi**2
      right_sum2 -= yi**2
      if i<self.min_left or xi == sort_x[i+1]:
        continue
      
      left_std = std_agg(left_count, left_sum, left_sum2)
      right_std = std_agg(right_count, right_sum, right_sum2)
      curr_score = left_std * left_count + right_std * right_count
      if curr_score < self.score:
        self.var_idx, self.score, self.split = var_idx, curr_score, xi

  @property
  def split_name(self):
    return self.x.columns[self.var_idx]

  @property
  def split_col(self):
    return self.x.values[self.idxs, self.var_idx]

  @property
  def is_left(self):
    return self.score == float('inf')

  def __repr__(self):
    s = f'n: {self.n}; val:{self.val}'
    if not self.is_leaf:
      s += f'; score:{self.score}; split:{self.split}; var{self.split_name}'
    return s

  def predict(self, x):
    return np.array([self.predict_row(xi) for xi in x])

  def predict_row(self, xi):
    if self.is_leaf: 
      return self.val
    t = self.left_subtree if xi[self.var_idx] <= self.split else self.right_subtree
    return t.predict_row(xi)